[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/workshop-julia-for-kaggle/main?filepath=introduction.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Call me Stephan

Hi there, I am Stephan Sahm, founder of [Jolin.io](www.jolin.io) consulting and organizer of the [Julia User Group Munich](https://www.meetup.com/julia-user-group-munich/).

We at [Jolin.io](www.jolin.io) bring Julia into production.

# Why Julia

- **R** is lovely for statistics and reports, the language and the whole community is build for it and has excellent support
- **Python** is your tool if you want to have things done, it has the largest package ecosystem of all languages
- **Julia** is made for high performance computing

Think of Julia as the new Fortran
- excellent runtime performance (like Fortran)
- high level language (actually Fortran also tries to be high-level, it just doesn't match today's standards)

# Julia 101

Julia is really simple

In [ ]:
array = [
    1 2
    3 4
]

In [ ]:
bigarray = [
    array        array .* 10
    1 ./ array   zeros(2, 2)
]

In [ ]:
function myfunction(x)
    if x > 3
        "big"
    else
        "small"
    end
end

In [ ]:
myfunction.(bigarray)

## Julia's top ingredients 1/2: `structs`

composed data types with speed like c

In [ ]:
struct Cat
    name
end

struct Dog
    # :: type annotations
    name::String
    age::Int
end

In [ ]:
🐱 = Cat("kitty")
🐕 = Dog("dexter", 2)

🐱.name, 🐕.name, 🐕.age

## Julia's top ingredients 2/2: `functions`

overloadable functions with runtime dispatch

In [ ]:
describe(animal) = "This is $(animal.name)"

describe(🐕)

In [ ]:
meet(animal1, animal2) = "$(animal1.name) meets $(animal2.name)"

meet(🐕, 🐱)

In [ ]:
function our_little_story(animal1, animal2)
    return [
        describe(animal1)
        describe(animal2)
        meet(animal1, animal2)
    ]
end

our_little_story(🐱, 🐕)

In [ ]:
describe(dog::Dog) = "This is $(dog.name) and it is $(dog.age) years old"

meet(cat::Cat, dog::Dog) = "The awesome cat $(cat.name) meets our $(dog.age) years old dog $(dog.name)."

our_little_story(🐱, 🐕)

### A little challenge for you

Define a duck 🦆 and tell a story about your duck meeting kitty 🐱 or dexter 🐕, your choice

In [ ]:
# your space
# ...

# Julia Deep Learning

The whole julia language is auto-differentiable.


Here a cpu version of the [quickstart example](https://fluxml.ai/Flux.jl/stable/models/quickstart/) from [Flux.jl](https://github.com/FluxML/Flux.jl)

In [ ]:
using Flux, Statistics, ProgressMeter, Plots

# Generate some data for the XOR problem: vectors of length 2, as columns of a matrix:
noisy = rand(Float32, 2, 1000)                                    # 2×1000 Matrix{Float32}
truth = [xor(col[1]>0.5, col[2]>0.5) for col in eachcol(noisy)]   # 1000-element Vector{Bool}

scatter(noisy[1,:], noisy[2,:], zcolor=truth, title="True classification", legend=false)

In [ ]:
model = Chain(
    Dense(2 => 3, tanh),   # activation function inside layer
    BatchNorm(3),
    Dense(3 => 2),
    softmax,
)

# The model encapsulates parameters, randomly initialised. Its initial output is:
out1 = model(noisy)
scatter(noisy[1,:], noisy[2,:], zcolor=out1[1,:], title="Untrained network", label="", clims=(0,1))

In [ ]:
# To train the model, we use batches of 64 samples, and one-hot encoding:
target = Flux.onehotbatch(truth, [true, false]) # 2×1000 OneHotMatrix
loader = Flux.DataLoader((noisy, target), batchsize=64, shuffle=true);
# 16-element DataLoader with first element: (2×64 Matrix{Float32}, 2×64 OneHotMatrix)

losses = []  # will store training progress
optim = Flux.setup(Flux.Adam(0.01), model)  # will store optimiser momentum, etc.

In [ ]:
# Training loop, using the whole data set 1000 times:
@showprogress for epoch in 1:1_000
    for (x, y) in loader
        loss, grads = Flux.withgradient(model) do m
            # Evaluate model and loss inside gradient context:
            y_hat = m(x)
            Flux.crossentropy(y_hat, y)
        end
        Flux.update!(optim, model, grads[1])
        push!(losses, loss)  # logging, outside gradient context
    end
end

optim # parameters, momenta and output have all changed

plot(losses; xaxis=(:log10, "iteration"), yaxis="loss", label="per batch")
n = length(loader)
plot!(n:n:length(losses), mean.(Iterators.partition(losses, n)), label="epoch mean", dpi=200)

In [ ]:
out2 = model(noisy)
scatter(noisy[1,:], noisy[2,:], zcolor=out2[1,:], title="Trained network", legend=false)

In [ ]:
accuracy(out) = mean((out[1,:] .> 0.5) .== truth)
accuracy(out1), accuracy(out2)

Further details about the Flux ecosystem can be found at [fluxml.ai](https://fluxml.ai/). 
To define your own Flux layers in julia, checkout [this tutorial](https://fluxml.ai/Flux.jl/stable/models/basics/).

# Thank you

Next is [2. How to use Julia directly at Kaggle](https://www.kaggle.com/stephansahm/titanic-tutorial-julia-version).

If you have any questions, you can reach me at stephan.sahm@jolin.io